In [1]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, RealAmplitudes
import matplotlib.pyplot as plt
import numpy as np

from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit import Aer, QuantumCircuit

from qiskit_machine_learning.neural_networks import EffectiveDimension, LocalEffectiveDimension

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit.algorithms.optimizers import COBYLA

from IPython.display import clear_output

In [2]:
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook
from pennylane import numpy as np
import qiskit
import pennylane as qml
import random as rand
import time as time
import pandas as pd
import os as os
from qiskit import Aer
from qiskit.utils import QuantumInstance
from sklearn.preprocessing import normalize

In [3]:
n_wires = 4
layers = 4
dev = qml.device('default.qubit', wires=4)
paths = ['iris.data', 'banknote.data']
DATAFRAMES = [((os.path.splitext(os.path.basename(path))[0]).capitalize(), pd.read_csv(
    path, names=['a0', 'a1', 'a2', 'aNUM_ITERATIONS', 'target'], header = 0)) for path in paths]

@qml.qnode(dev)

def pennylane_circuit(features):
    qml.AngleEmbedding(features=features, wires=range(4), rotation='Z')
    qml.Rot(features[0], features[0], features[0], wires=0)
    qml.Rot(features[1], features[1], features[1], wires=1)
    qml.Rot(features[2], features[2], features[2], wires=2)
    qml.Rot(features[3], features[3], features[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

    return qml.expval(qml.PauliZ(0))

# https://quantaggle.com/algorithms/ansatz/
    
@qml.qnode(dev)
def hardware_efficient_circuit(W):
    qml.AngleEmbedding(features=W, wires=range(4), rotation='Z')

    qml.RY(W[0], wires=0)
    qml.RY(W[1], wires=1)
    qml.RY(W[2], wires=2)
    qml.RY(W[3], wires=3)
    qml.RZ(W[0], wires=0)
    qml.RZ(W[1], wires=1)
    qml.RZ(W[2], wires=2)
    qml.RZ(W[3], wires=3)
    qml.CZ(wires=[0, 1])
    qml.CZ(wires=[1, 2])
    qml.CZ(wires=[2, 3])
    qml.CZ(wires=[3, 0])

    qml.RY(W[0], wires=0)
    qml.RY(W[1], wires=1)
    qml.RY(W[2], wires=2)
    qml.RY(W[3], wires=3)
    qml.RZ(W[0], wires=0)
    qml.RZ(W[1], wires=1)
    qml.RZ(W[2], wires=2)
    qml.RZ(W[3], wires=3)
    
    return qml.expval(qml.PauliZ(0))

# https://pennylane.ai/qml/glossary/circuit_ansatz.html#a-parametrized-b-parametrized


@qml.qnode(dev)
def pennylane_ab_paramaterized(W):
    qml.AngleEmbedding(features=W, wires=range(4), rotation='Z')
    qml.CRot(W[0], W[0], W[0], wires=[0, 2])
    qml.CRot(W[1], W[1], W[1], wires=[1, 3])
    qml.CRot(W[2], W[2], W[2], wires=[2, 0])
    qml.CRot(W[3], W[3], W[3], wires=[3, 1])

    return qml.expval(qml.PauliZ(0))

# https://arxiv.org/pdf/1612.02806.pdf


@qml.qnode(dev)
def pairwise_controlled_rot(W):
    qml.AngleEmbedding(features=W, wires=range(4), rotation='Z')
    qml.CRot(W[0], W[0], W[0],wires=[0, 1])
    qml.CRot(W[0], W[0], W[0],wires=[0, 2])
    qml.CRot(W[0], W[0], W[0],wires=[0, 3])

    qml.CRot(W[1], W[1], W[1],wires=[1, 0])
    qml.CRot(W[1], W[1], W[1],wires=[1, 2])
    qml.CRot(W[1], W[1], W[1],wires=[1, 3])

    qml.CRot(W[2], W[2], W[2],wires=[1, 0])
    qml.CRot(W[2], W[2], W[2],wires=[1, 2])
    qml.CRot(W[2], W[2], W[2],wires=[1, 3])
    
    qml.CRot(W[3], W[3], W[3],wires=[1, 0])
    qml.CRot(W[3], W[3], W[3],wires=[1, 2])
    qml.CRot(W[3], W[3], W[3],wires=[1, 3])

    return qml.expval(qml.PauliZ(0))

#https://arxiv.org/pdf/2201.04093.pdf FSIM implementation

iris = np.array([[5.1,3.5,1.4,0.2],
[4.9,3.0,1.4,0.2],
[4.7,3.2,1.3,0.2],
[4.6,3.1,1.5,0.2],
[5.0,3.6,1.4,0.2],
[5.4,3.9,1.7,0.4],
[4.6,3.4,1.4,0.3],
[5.0,3.4,1.5,0.2],
[4.4,2.9,1.4,0.2],
[4.9,3.1,1.5,0.1],
[5.4,3.7,1.5,0.2],
[4.8,3.4,1.6,0.2]])

iris2 = np.array([[5.1,3.5,1.4,0.2],
[4.9,3.0,1.4,0.2],
[4.7,3.2,1.3,0.2],
[4.6,3.1,1.5,0.2],
[5.0,3.6,1.4,0.2],
[5.4,3.9,1.7,0.4],
[4.6,3.4,1.4,0.3],
[5.0,3.4,1.5,0.2],
[4.4,2.9,1.4,0.2],
[4.9,3.1,1.5,0.1],
[5.4,3.7,1.5,0.2],
[4.8,3.4,1.6,0.2],
[4.8,3.0,1.4,0.1],
[4.3,3.0,1.1,0.1],
[5.8,4.0,1.2,0.2],
[5.7,4.4,1.5,0.4],
[5.4,3.9,1.3,0.4],
[5.1,3.5,1.4,0.3],
[5.7,3.8,1.7,0.3],
[5.1,3.8,1.5,0.3],
[5.4,3.4,1.7,0.2],
[5.1,3.7,1.5,0.4],
[4.6,3.6,1.0,0.2],
[5.1,3.3,1.7,0.5],
[4.8,3.4,1.9,0.2],
[5.0,3.0,1.6,0.2],
[5.0,3.4,1.6,0.4],
[5.2,3.5,1.5,0.2],
[5.2,3.4,1.4,0.2],
[4.7,3.2,1.6,0.2],
[4.8,3.1,1.6,0.2],
[5.4,3.4,1.5,0.4],
[5.2,4.1,1.5,0.1],
[5.5,4.2,1.4,0.2],
[4.9,3.1,1.5,0.1],
[5.0,3.2,1.2,0.2],
[5.5,3.5,1.3,0.2],
[4.9,3.1,1.5,0.1],
[4.4,3.0,1.3,0.2],
[5.1,3.4,1.5,0.2],
[5.0,3.5,1.3,0.3],
[4.5,2.3,1.3,0.3],
[4.4,3.2,1.3,0.2],
[7.0,3.2,4.7,1.4,],
[6.4,3.2,4.5,1.5],
[6.9,3.1,4.9,1.5],
[5.5,2.3,4.0,1.3],
[6.5,2.8,4.6,1.5],
[5.7,2.8,4.5,1.3],
[6.3,3.3,4.7,1.6],
[4.9,2.4,3.3,1.0],
[6.6,2.9,4.6,1.3],
[5.2,2.7,3.9,1.4],
[5.0,2.0,3.5,1.0],
[5.9,3.0,4.2,1.5],
[6.0,2.2,4.0,1.0]])
                 
                 
                 

@qml.qnode(dev)
def circuit(params):
    qml.AngleEmbedding(features=params, wires=range(4), rotation='Z')
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)
    qml.CNOT(wires=(1,0))
    qml.CNOT(wires=(1,2))
    qml.CNOT(wires=(2,3))
    qml.CNOT(wires=(3,0))
    return qml.expval(qml.PauliZ(0))

circuit_lib = [circuit, pennylane_circuit, hardware_efficient_circuit,
                    pennylane_ab_paramaterized, pairwise_controlled_rot]

for circuit in circuit_lib:
    mt_fn = qml.metric_tensor(circuit, approx="diag")
    print(mt_fn(iris2[0]))

[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
[[0.46428321 0.         0.         0.        ]
 [0.         0.28076222 0.         0.        ]
 [0.         0.         0.49277779 0.        ]
 [0.         0.         0.         0.25986738]]
[[0.76112539 0.         0.         0.        ]
 [0.         0.58522612 0.         0.        ]
 [0.         0.         0.78605064 0.        ]
 [0.         0.         0.         0.53040304]]
[[0.125 0.    0.    0.   ]
 [0.    0.125 0.    0.   ]
 [0.    0.    0.125 0.   ]
 [0.    0.    0.    0.125]]
[[0.375 0.    0.    0.   ]
 [0.    0.375 0.    0.   ]
 [0.    0.    0.375 0.   ]
 [0.    0.    0.    0.375]]
